# Извлечение данных из PDF с помощью Docling

In [ ]:
# Установка Docling и зависимостей
!pip install docling pandas tabulate

In [ ]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
import pandas as pd
import json

# Параметры
pdf_path = "kztkf5m1_2025_cons_rus_pdf.pdf"
pages_to_extract = [11, 15]

print(f"PDF файл: {pdf_path}")
print(f"Страницы для извлечения: {pages_to_extract}")

In [ ]:
# Настройка конвертера с опциями для PDF
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True  # Включаем OCR
pipeline_options.do_table_structure = True  # Распознавание структуры таблиц

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

print("Конвертер настроен с OCR и распознаванием таблиц")

In [ ]:
# Конвертация PDF
print("Конвертация PDF... (это может занять несколько минут)")
conv_result = converter.convert(pdf_path)
doc = conv_result.document
print("Конвертация завершена!")
print(f"Найдено таблиц в документе: {len(doc.tables)}")

In [ ]:
# Извлечение таблиц по страницам
extracted_data = {
    "source_file": pdf_path,
    "pages_requested": pages_to_extract,
    "extracted_pages": {}
}

for page_num in pages_to_extract:
    page_tables = []
    page_text = []
    
    # Извлекаем таблицы для данной страницы
    for table_ix, table in enumerate(doc.tables):
        # Проверяем номер страницы таблицы
        if hasattr(table, 'prov') and table.prov:
            for prov in table.prov:
                if hasattr(prov, 'page_no') and prov.page_no == page_num:
                    # Экспортируем таблицу в DataFrame
                    try:
                        df = table.export_to_dataframe(doc=doc)
                        table_data = {
                            "table_index": table_ix,
                            "rows": len(df),
                            "columns": len(df.columns),
                            "headers": df.columns.tolist(),
                            "data": df.to_dict(orient='records'),
                            "markdown": df.to_markdown(index=False),
                            "html": table.export_to_html(doc=doc)
                        }
                        page_tables.append(table_data)
                    except Exception as e:
                        page_tables.append({
                            "table_index": table_ix,
                            "error": str(e)
                        })
                    break
    
    # Извлекаем текстовые элементы для данной страницы
    for item, level in doc.iterate_items():
        if hasattr(item, 'prov') and item.prov:
            for prov in item.prov:
                if hasattr(prov, 'page_no') and prov.page_no == page_num:
                    if hasattr(item, 'text') and item.text:
                        page_text.append({
                            "type": type(item).__name__,
                            "text": item.text
                        })
                    break
    
    extracted_data["extracted_pages"][f"page_{page_num}"] = {
        "page_number": page_num,
        "tables_count": len(page_tables),
        "tables": page_tables,
        "text_elements_count": len(page_text),
        "text_elements": page_text,
        "full_text": "\n".join([item["text"] for item in page_text])
    }

print(f"Обработано страниц: {len(extracted_data['extracted_pages'])}")
for page_key, page_data in extracted_data['extracted_pages'].items():
    print(f"  {page_key}: {page_data['tables_count']} таблиц, {page_data['text_elements_count']} текстовых элементов")

In [ ]:
# Вывод таблиц в красивом формате
print("=" * 60)
print("ИЗВЛЕЧЕННЫЕ ТАБЛИЦЫ:")
print("=" * 60)

for page_num in pages_to_extract:
    page_data = extracted_data["extracted_pages"].get(f"page_{page_num}")
    if page_data and page_data["tables"]:
        print(f"\n{'='*20} Страница {page_num} {'='*20}")
        for i, table in enumerate(page_data["tables"]):
            print(f"\n--- Таблица {i+1} ({table.get('rows', '?')} строк x {table.get('columns', '?')} столбцов) ---\n")
            if "markdown" in table:
                print(table["markdown"])
            elif "error" in table:
                print(f"Ошибка: {table['error']}")
    else:
        print(f"\nСтраница {page_num}: таблиц не найдено")

In [ ]:
# Вывод JSON результата (без HTML для читаемости)
result_for_json = extracted_data.copy()
for page_key in result_for_json["extracted_pages"]:
    for table in result_for_json["extracted_pages"][page_key]["tables"]:
        if "html" in table:
            table["html"] = "[HTML content - see separate output]"
        if "markdown" in table:
            table["markdown"] = "[Markdown content - see separate output]"

print(json.dumps(result_for_json, ensure_ascii=False, indent=2))

In [ ]:
# Сохранение полного результата в файл
with open("extracted_data_docling.json", "w", encoding="utf-8") as f:
    json.dump(extracted_data, f, ensure_ascii=False, indent=2)

print("Результат сохранен в extracted_data_docling.json")

In [ ]:
# Сохранение таблиц в CSV файлы
for page_num in pages_to_extract:
    page_data = extracted_data["extracted_pages"].get(f"page_{page_num}")
    if page_data and page_data["tables"]:
        for i, table in enumerate(page_data["tables"]):
            if "data" in table and table["data"]:
                df = pd.DataFrame(table["data"])
                filename = f"table_page{page_num}_{i+1}.csv"
                df.to_csv(filename, index=False, encoding="utf-8-sig")
                print(f"Сохранено: {filename}")

In [ ]:
# Вывод текста со страниц
print("=" * 60)
print("ИЗВЛЕЧЕННЫЙ ТЕКСТ:")
print("=" * 60)

for page_num in pages_to_extract:
    page_data = extracted_data["extracted_pages"].get(f"page_{page_num}")
    if page_data:
        print(f"\n{'='*20} Страница {page_num} {'='*20}\n")
        print(page_data["full_text"])